## data creating

In [1]:
import json
import pandas as pd

# Define the path to your JSONL file
jsonl_file_path = 'data.jsonl'
# Define the path to the output CSV file
csv_file_path = 'output.csv'

# Read the JSONL file and store the data in a list
data = []
with open(jsonl_file_path, 'r') as file:
    for line in file:
        data.append(json.loads(line))

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(data)

# Rename columns to match the required CSV format
df.columns = ['input_text', 'response_text']

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

print(f"Data successfully converted to {csv_file_path}")


Data successfully converted to output.csv


In [2]:
import pandas as pd
from datasets import load_dataset, Dataset

# Load the CSV file into a pandas DataFrame
df = pd.read_csv('output.csv')

# Display the DataFrame to check if the data is loaded correctly
print(df.head())

# Convert the DataFrame to a Dataset
dataset = Dataset.from_pandas(df)

# Display the Dataset to verify the conversion
print(dataset)

# You can now use this dataset to train your model.
# For example, to save the dataset to a file:
dataset.to_csv('prepared_customer_seller_data.csv', index=False)

# To load the dataset later:
loaded_dataset = load_dataset('csv', data_files='prepared_customer_seller_data.csv')

# Display the loaded dataset to verify
print(loaded_dataset)


c:\Users\mithi\miniconda3\envs\face_rec\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


                                     input_text  \
0                  What is the price of the TV?   
1                  Do you have any gas cookers?   
2                              I need a fridge.   
3  Can you tell me about your Samsung products?   
4     Do you have any discounts on electronics?   

                                       response_text  
0       Sure, let me check the latest price for you.  
1  Sure, let me see what gas cookers we have in s...  
2        Let me check the available fridges for you.  
3  I can provide you with the latest details on o...  
4  Let me check if we have any discounts on elect...  
Dataset({
    features: ['input_text', 'response_text'],
    num_rows: 71
})


Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 60.72ba/s]
Generating train split: 71 examples [00:00, 5065.15 examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_text', 'response_text'],
        num_rows: 71
    })
})


## training model

In [15]:
import pandas as pd
from datasets import Dataset
from transformers import Trainer, TrainingArguments, GPT2LMHeadModel, GPT2Tokenizer

# Use the correct file path
file_path = 'prepared_customer_seller_data.csv'

try:
    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Combine input and response text for causal language modeling
    df['text'] = df['input_text'] + " " + df['response_text']
    
    # Convert the DataFrame to a Dataset
    dataset = Dataset.from_pandas(df[['text']])
    
    # Initialize the tokenizer and model
    model_name = 'gpt2'  # Change to 'gpt2-small' if needed
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    
    # Add a padding token if not already present
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token  # Use the end-of-sequence token as padding token

    # Tokenize the combined text
    def tokenize_function(example):
        return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)
    
    tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
    
    # Prepare training arguments
    training_args = TrainingArguments(
        output_dir='./results',
        per_device_train_batch_size=1,
        num_train_epochs=3,  # Adjust the number of epochs as needed
        logging_dir='./logs',
        logging_steps=10,
        save_steps=500,
        save_total_limit=2,
    )
    
    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets,
    )
    
    # Train the model
    trainer.train()
    
    # Save the model
    model.save_pretrained('custom_seller_bot_model')
    tokenizer.save_pretrained('custom_seller_bot_model')
    
    print("Training complete and model saved.")
    
except FileNotFoundError:
    print(f"The file at {file_path} was not found. Please check the path and try again.")
except Exception as e:
    print(f"An error occurred: {e}")


Map: 100%|██████████| 71/71 [00:00<00:00, 2268.49 examples/s]

An error occurred: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`


In [6]:
import pandas as pd
from datasets import Dataset
from transformers import Trainer, TrainingArguments, GPT2LMHeadModel, GPT2Tokenizer

# Use the correct file path
file_path = 'prepared_customer_seller_data.csv'

try:
    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Combine input and response text for causal language modeling
    df['text'] = df['input_text'] + " " + df['response_text']
    
    # Convert the DataFrame to a Dataset
    dataset = Dataset.from_pandas(df[['text']])
    
    # Initialize the tokenizer and model
    model_name = 'gpt2'  # or 'gpt2-small', 'gpt2-medium', 'gpt2-large', etc.
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    
    # Add a padding token if not already present
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token  # Use the end-of-sequence token as padding token

    # Tokenize the combined text
    def tokenize_function(example):
        encodings = tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)
        encodings["labels"] = encodings["input_ids"].copy()  # Use input_ids as labels
        return encodings
    
    tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
    
    # Prepare training arguments
    training_args = TrainingArguments(
        output_dir='./results',
        per_device_train_batch_size=1,
        num_train_epochs=3,
        logging_dir='./logs',
        logging_steps=10,
        save_steps=500,
        save_total_limit=2,
        report_to=None,  # Ensure you're not using any unsupported reporting options
    )
    
    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets,
    )
    
    # Train the model
    trainer.train()
    
    # Save the model
    model.save_pretrained('custom_seller_bot_model')
    tokenizer.save_pretrained('custom_seller_bot_model')
    
    print("Training complete and model saved.")
    
except FileNotFoundError:
    print(f"The file at {file_path} was not found. Please check the path and try again.")
except Exception as e:
    print(f"An error occurred: {e}")


  5%|▍         | 10/213 [03:08<36:24, 10.76s/it] 

{'loss': 2.2613, 'grad_norm': 2.626744270324707, 'learning_rate': 4.765258215962441e-05, 'epoch': 0.14}


  9%|▉         | 20/213 [04:33<27:38,  8.59s/it]

{'loss': 0.0983, 'grad_norm': 1.6586312055587769, 'learning_rate': 4.530516431924883e-05, 'epoch': 0.28}


 14%|█▍        | 30/213 [06:07<29:19,  9.61s/it]

...

{'loss': 0.013, 'grad_norm': 0.6004200577735901, 'learning_rate': 3.051643192488263e-06, 'epoch': 2.82}


 99%|█████████▊| 210/213 [34:09<00:24,  8.12s/it]

{'loss': 0.0192, 'grad_norm': 0.5160269737243652, 'learning_rate': 7.042253521126761e-07, 'epoch': 2.96}


100%|██████████| 213/213 [35:02<00:00,  9.87s/it]


{'train_runtime': 2102.3276, 'train_samples_per_second': 0.101, 'train_steps_per_second': 0.101, 'train_loss': 0.13021979012259854, 'epoch': 3.0}
Training complete and model saved.


In [13]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Check PyTorch version
print(f"PyTorch version: {torch.__version__}")

# Initialize model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

print("Model and tokenizer loaded successfully.")


PyTorch version: 2.3.1+cpu
Model and tokenizer loaded successfully.


In [16]:
import accelerate
import transformers

print(f"Accelerate version: {accelerate.__version__}")
print(f"Transformers version: {transformers.__version__}")


Accelerate version: 0.33.0
Transformers version: 4.43.4


In [5]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = 'gpt2'  # or 'gpt2-small'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
